# Special Method & Overload 

In [1]:
class Mylist(object):
    
    def __init__(self, *a): # extra input -> make list
        self.a = list(a)
    
    def __len__(self): # Take length of list
        return len(self.a)
    
    def __setitem__(self, key, value): # instance[key] = value then save instance.a[key] = 2 * value
        self.a[key] = 2*value
        
    def __getitem__(self, key): # instance[key] = 2 * self.a[key]
        return 3 * self.a[key]

In [2]:
b = Mylist(1, 2, 3, 4)
print b.a
print b.a[1]
print b[1]

[1, 2, 3, 4]
2
6


In [35]:
b.a[1] = 7
print b.a
print b.a[1]
print b[1]

[1, 7, 3, 4]
7
14


In [3]:
b[0] = 2
print b.a
print b.a[0]
print b[0]

[4, 2, 3, 4]
4
12


# Financial Transaction 

In [76]:
from datetime import date
from math import exp

today = date.today()
r_free = 0.05 / 365.0

class FinancialTransaction(object):
    
    def __init__(self, t, a, description = ''):
        self.t = t
        self.a = a
        self.description = description
    
    def pv(self, t0 = today, r = r_free):
        # date 생성자는 연도, 월, 날짜를 갖고있음. days는 일수를 불러옴.
        # t일 후에 발생하는 거래에서 self.a의 현재 매입 가치. t가 크면 클수록 줄어듬
        return self.a * exp(r * (t0 - self.t).days) 

In [77]:
class CashFlow(object):
    
    def __init__(self):
        self.transactions = []
        
    def add(self, transaction):
        self.transactions.append(transaction) # 이때, transaction은 FinancialTransaction이 instance를 넣을 것임.
    
    def pv(self, t0, r = r_free):
        # self.transactions에 들어있는 모든 transaction의 현재매입가치를 합산한다.
        return sum(x.pv(t0, r) for x in self.transactions) # x는 FinancialTransaction의 instance.
    
    def __getitem__(self, key):
        return self.transactions[key]

In [78]:
bond = CashFlow()
today = date(2012, 1, 1)

In [79]:
for year in range(2012, 2014):
    for month in range(1, 13):
        coupon = FinancialTransaction(date(year, month, 20), 1000) # 매월 20일 1000달러 지급 채권.
        bond.add(coupon) # 매월마다 나오는 채권을 bond에 추가.

print round(bond.pv(today, r = 0.05 / 365), 0) # 즉, 2년간 매월 20일 발행되는 채권들의 현재 매입 가치.

22827.0


* 따라서 1000달러씩 24개월은 원래 24000원이지만 연이율 5%이므로 현재 매입가치는 22827원이 된다. 

# URLlib & JSON

In [80]:
import urllib
page = urllib.urlopen('http://www.google.com')
html = page.read()

In [81]:
html

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script>(function(){window.google={kEI:\'8wG9V6SNBIbK0gTwyIKABw\',kEXPI:\'1351569,3700321,3700389,4029815,4031109,4032677,4036509,4036527,4038012,4039268,4041303,4043492,4045841,4048347,4052304,4061155,4062706,4063879,4064815,4065787,4065793,4066654,4066708,4067175,4067342,4068471,4068550,4068816,4068844,4069838,4069841,4069906,4070060,4070127,4070142,4070316,4070329,4070454,4070598,4070816,4071231,4071577,4071604,4071842,4072000,4072213,4072289,4072364,4072377,4072624,4072654,4072682,4072776,4073114,4073229,4073405,4073409,4073452,4073889,4073945,4073959,4073978,4074349,4074801,4075121,4075781,4075860,4075966,4075974,4076018,4076096,4076116,4076743,4077221,4077280,8300272,8502184,8503585,8504845,8504930,8505127,8505150,

In [82]:
class MyClass(object):
    pass

myinstance = MyClass()
myinstance.x = 'something'
a = [1, 2, {'hello': 'world'}, [3, 4, [myinstance]]]

In [83]:
import cPickle as pickle
b = pickle.dumps(a)
c = pickle.loads(b)

In [86]:
print b

(lp1
I1
aI2
a(dp2
S'hello'
p3
S'world'
p4
sa(lp5
I3
aI4
a(lp6
ccopy_reg
_reconstructor
p7
(c__main__
MyClass
p8
c__builtin__
object
p9
NtRp10
(dp11
S'x'
S'something'
p12
sbaaa.


In [87]:
print c

[1, 2, {'hello': 'world'}, [3, 4, [<__main__.MyClass object at 0x7fd9cc2522d0>]]]


# SQLite 

* 사전 자료형은 RAM에 일시적으로 저장되기 때문에 지속성이 결여.
* shelve 모듈은 Disk에 저장되지만 잠금 불가.
* 데이터베이스 사용이 적절한 대책 -> 파이썬에서는 SQLite라는 내장된 SQL데이터 베이스 존재.

In [90]:
import sqlite3

In [91]:
sqlite3.version

'2.6.0'

In [92]:
sqlite3.sqlite_version

'3.8.11.1'

## Connect to DB

In [93]:
con = sqlite3.connect("kospi.db")
type(con)

sqlite3.Connection

## Input data to DB

1. sqlite3.connect 메소드를 이용해서 DB 파일에 연결한 후 'Connection' 객체를 생성한다.
2. Connection객체를 통해 Cursor 객체를 생성한다.
3. 'Cursor' 객체의execute 메소드를 통해서 query를 실행한다.
4. 'Connection' 객체의 commit를 이용하여 변경된 내용을 commit한다.
5. DB와의 연결을 닫는다.


In [95]:
cursor = con.cursor() # cursor object

* Before input data, should make table!

In [100]:
cursor.execute("CREATE TABLE kospi(Name text, ClosingPrice int)")

ProgrammingError: Cannot operate on a closed database.

* Input Data 

In [98]:
cursor.execute("INSERT INTO kospi VALUES('LG전자', 74500)")
cursor.execute("INSERT INTO kospi VALUES('네이버', 774000)")
cursor.execute("INSERT INTO kospi VALUES('다음', 169100)")

In [99]:
con.commit()
con.close()

## From DB read Data

In [107]:
con = sqlite3.connect("kospi.db")
cursor = con.cursor()

In [108]:
cursor.execute("SELECT * FROM kospi") # kospi 테이블로부터 모든것(*)을 선택

In [110]:
for row in cursor:
    print row

(u'\ub124\uc774\ubc84', 774000)
(u'\ub2e4\uc74c', 169100)


In [111]:
cursor.execute("SELECT * FROM kospi")
cursor.fetchone()

(u'LG\uc804\uc790', 74500)

In [112]:
cursor.fetchone()

(u'\ub124\uc774\ubc84', 774000)

In [113]:
cursor.fetchone()

(u'\ub2e4\uc74c', 169100)

In [114]:
cursor.fetchone()

In [115]:
cursor.execute("SELECT * FROM kospi")

In [116]:
cursor.fetchall()

[(u'LG\uc804\uc790', 74500),
 (u'\ub124\uc774\ubc84', 774000),
 (u'\ub2e4\uc74c', 169100)]